In [1]:
import pandas as pd
import numpy as np

In [2]:
targets = pd.read_csv("../../../../data/tp2/target_competencia_ids.csv")

In [3]:
targets["ref_hash"] = targets["ref_hash"].map(lambda x: x[:-3])

In [4]:
targets["ref_hash"] = targets["ref_hash"].astype("int64")

---

### Features auctions

In [5]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")


### Me quedo con datos del 24 en adelante

In [6]:
fecha_inicio = "2019-04-24"

In [7]:
auctions = auctions[auctions["date"] > fecha_inicio]

In [8]:
auctions.sort_values(by = "date",inplace = True)

In [9]:
targets_individuales = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

## Cantidad de apariciones discriminando períodos

In [10]:
# Creo columnas para luego separar
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day

---

#### Por hora

In [11]:
feature = auctions.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["apariciones_en_hora_{}".format(hora) for hora in range(0,24)]

In [12]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [13]:
targets_individuales.isnull().sum().max()

686

##### Hay 686 dispositivos sin datos de auctions

---

#### Por dia

In [14]:
feature = auctions.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["apariciones_en_dia_{}".format(dia) for dia in range(1,4)]


In [15]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Sistema operativo (Android / iOS)

#### Cantidad de so

In [16]:
feature=  auctions.groupby("device_id").agg({"ref_type_id" : "nunique"}).rename(columns={"ref_type_id": "cantidad_so_registrados"})

In [17]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

#### Media so

In [18]:
feature = auctions.groupby("device_id").agg({"ref_type_id" : "mean"}).rename(columns={"ref_type_id": "so_media"})

In [19]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Tiempo medio entre apariciones

In [20]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
feature = pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_medio_entre_apariciones"})

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Tiempo máximo entre apariciones

In [ ]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).max()
feature = pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_maximo_entre_apariciones"})

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Desvío entre apariciones

In [ ]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).std()
feature=  pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "desvio_entre_apariciones"})

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Cantidad de apariciones por fuente

In [ ]:
feature = auctions.groupby(["device_id", "source_id"])["date"].agg("count").to_frame().unstack()
feature.columns = ["apariciones_en_source_{}".format(x) for x in range(10)]
feature = feature.fillna(0).astype(np.int64)

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Tiempo desde la última aparición hasta el fin de la ventana

In [ ]:
tiempo_final = "2019-04-27"

In [ ]:
feature =(np.datetime64(tiempo_final) - auctions.groupby("device_id")\
          .agg({"date" : "max"})["date"]).dt.total_seconds().to_frame()\
        .rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana"})

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Tiempo desde t0 hasta horas más frecuentes

#### Funciones para evitar codigo repetido

In [ ]:
def obtener_horas_aparicion_maxima(dataframe):
    """Devuelve un dataframe con la pinta {'device_id','hora','date'} donde cada dispositivo
    contiene todas las horas donde su aparición en subastas sea igual a la maxima """
    
    feature = dataframe.groupby(["device_id","hora"]).agg({"date":"count"})
    idx = feature.groupby(["device_id"])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    feature.reset_index(inplace = True)
    return feature

In [ ]:
def obtener_tiempo_hasta_hora_minima(feature):
    """Recibe un dataframe feature con horas de aparicion máxima y devuelve el
    tiempo hasta la hora minima para cada dispositivo"""
    segundos_en_hora = 60*60

    feature = feature.reset_index().groupby("device_id").agg({"hora":"min"})
    feature["hora"] = feature["hora"] * segundos_en_hora
    return feature
    

---

## Tiempo desde t0 hasta hora más frecuente de aparicion

#### Frente a empate entre horas me quedaré con la minima (la mas cercana a t0)

In [ ]:
auctions["hora"] = auctions["date"].dt.hour


#### Tiempo hasta hora minima

In [ ]:
segundos_en_hora = 60*60
    
feature = obtener_horas_aparicion_maxima(auctions)
    
    #Obtengo hora más cerca de t0 = 00 hs
feature = obtener_tiempo_hasta_hora_minima(feature)
feature.rename(columns = {"hora":"tiempo_hora_frec_aparicion_subasta"},inplace = True)
    


In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

## Tiempo desde t0 hasta la segunda hora más frecuente de aparición

In [ ]:
feature = auctions.groupby(["device_id","hora"]).agg({"date":"count"})
feature.reset_index(inplace = True)
    
    
apariciones_maximas = obtener_horas_aparicion_maxima(auctions)
apariciones_maximas.reset_index(inplace = True)

    
    
#Obtengo los indices de las horas minimas de aparicion maxima para cada dispositivo
horas_minimas = apariciones_maximas.groupby("device_id")["hora"].transform(min) == apariciones_maximas["hora"]

apariciones_maximas = apariciones_maximas[horas_minimas]

    
#Me quedo con los datos que no esten entre las horas minimas de aparicion maxima
horas_no_minimas = ~( (feature.isin(apariciones_maximas)))
#Todas las columnas deben ser True
horas_no_minimas = horas_no_minimas.all(1)
    
        
#Me quedo con los datos para completar aquellos dispositivos que solo contienen datos de una sola hora
datos_restantes = feature[ ~(horas_no_minimas)]

    
#Me quedo con los datos que excluyena las horas minimas 
feature = feature[horas_no_minimas]
    
#obtengo las apariciones maximas dentro de los valores restantes 
feature = obtener_horas_aparicion_maxima(feature)
    
#obtengo las horas minimas dentro de los valores restantes( segundas horas minimas)
feature = obtener_tiempo_hasta_hora_minima(feature)
        
#Me quedo con los datos que contienen solo una hora 
apariciones_unicas = ~( (datos_restantes["device_id"].isin(feature.index)))
    
datos_restantes = datos_restantes[apariciones_unicas]
  
datos_restantes = obtener_tiempo_hasta_hora_minima(datos_restantes)
     
feature = feature.append(datos_restantes)
    
feature.rename(columns = {"hora":"tiempo_segunda_hora_frec_aparicion"},inplace = True)


In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [ ]:
del auctions

----

# Events

In [ ]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [ ]:
events = events[events["date"] > fecha_inicio]

In [ ]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

In [ ]:
# Creo columnas para luego separar
events['dia'] = events['date'].dt.day
events.rename(columns = {"dayofweek":"dia_de_semana"},inplace = True)

### Cantidad de eventos por dia

In [ ]:
dia_final = 4

In [ ]:
feature = events.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["eventos_dia_{}".format(dia) for dia in range(1,dia_final)]
 

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

### Cantidad de eventos wifi- sin wifi

In [ ]:
events["sin_wifi"] = ~(events["wifi"])

In [ ]:
feature = events.groupby("device_id").agg({"wifi":"sum","sin_wifi":"sum"}).rename(columns={"wifi": "cant_eventos_wifi",
                                                                                              "sin_wifi":"cant_eventos_sin_wifi"})
    

In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [ ]:
del events

---

# Clicks

In [ ]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

In [ ]:
clicks = clicks[clicks["created"] > fecha_inicio]

### Cantidad de clicks por dia

In [ ]:
# Creo columnas para luego separar
clicks['dia'] = clicks['created'].dt.day

In [ ]:
dias = clicks["dia"].drop_duplicates
feature = clicks.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["clicks_en_dia_{}".format(dia) for dia in range(1,4)]


In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [ ]:
del clicks

---

## Installs

In [ ]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [ ]:
installs = installs[installs["created"] > fecha_inicio]

### Cantidad de instalaciones por dia

In [ ]:
installs['dia'] = installs['created'].dt.day


In [ ]:
dias = installs["dia"].drop_duplicates
feature = installs.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["instalaciones_dia_{}".format(dia) for dia in range(1,4)]


In [ ]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [ ]:
len(targets_individuales.columns)

---

## Guardado

In [ ]:
targets_individuales.to_pickle("../../features/features_preds_auctions")

In [ ]:
targets_individuales.isnull().sum()

In [ ]:
targets_individuales.columns